In [1]:
import pandas as pd
import numpy as np

## import training and test datasets

In [401]:
# pull in files and create dataframes
train = np.loadtxt("pa3train.txt")
test = np.loadtxt("pa3test.txt")
#dictionary = np.loadtxt("pa3dictionary.txt")


# training data 
df_train = pd.DataFrame(train)
df_train_2 = df_train[df_train.apply(lambda x: x[819] == 1 or x[819] == 2, axis=1)]
#df_train_2 = df_train_2.sample(frac=1)
# test data
df_test = pd.DataFrame(test)
df_test_2 = df_test[df_test.apply(lambda x: x[819] == 1 or x[819] == 2, axis=1)]
#df_dict = pd.DataFrame("pa3dictionary.txt")



X_train = df_train.drop([819], axis=1)
y_train = df_train[819]

X_train_2 = df_train_2.drop([819], axis=1)
y_train_2 = df_train_2[819]

## mapping 1 -> 1 and 2 -> -1
y_train_2_trans = y_train_2.apply(lambda x: 1 if x > 1 else -1)

X_test = df_test.drop([819], axis=1)
y_test = df_test[819]

X_test_2 = df_test_2.drop([819], axis=1)
y_test_2 = df_test_2[819]
# mapping 1 -> 1 and 2 -> -1
y_test_2_trans = y_test_2.apply(lambda x: 1 if x > 1 else -1)

label = 819

# Create Perceptron Class to train classifier

In [421]:
class Perceptron:
    def __init__(self, rounds):
        self.rounds = rounds
        self.X = None
        self.y = None
        self.w = None
        self.array_of_w = []
        self.predictions = []
        self.voted_predictions = []
        self.avg_predictions = []
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.w = pd.Series(np.arange(len(self.X.columns)))*0
        m = 1
        c = 1
        for j in range(self.rounds):
            for i in range(len(self.X)):
                if self.y.iloc[i]*self.w.dot(self.X.iloc[i]) <= 0:
                    self.w = self.w + self.y.iloc[i]*self.X.iloc[i]
                    w = self.w
                    m = m + 1 
                    c = 1
                    self.array_of_w.append([w, c])
                else: 
                    c = c + 1
                    index = len(self.array_of_w) - 1 # last position of last [w, c]
                    self.array_of_w[index][1] = c
    
    def perceptron_get_sign(self, w, x):
        return -1 if w.dot(x) <= 0 else 1
    
    def perceptron_get_sign_with_c(self, w, x, c):
        return c*self.perceptron_get_sign(w,x)
        
    def predict(self):
        for i in range(len(self.X)):
            x = self.X.iloc[i]
            self.predictions.append(self.perceptron_get_sign(self.w, x))
        return self.predictions
    
    def voted_predict(self):
        for i in range(len(self.X)):
            x = self.X.iloc[i]
            voted_sum = 0
            for w_c in self.array_of_w:
                w = w_c[0]
                c = w_c[1]
                voted_sum = voted_sum + self.perceptron_get_sign_with_c(w, x, c)

            pred = -1 if voted_sum <= 0 else 1
            self.voted_predictions.append(pred)

        return self.voted_predictions
    
    def avg_predict(self):
        # get first w,c pair
        w = self.array_of_w[0][0] # grab w
        c = self.array_of_w[0][1] # grab c
        w_avg = c*w # [0, ..., 0] + first c*w
        for i in range(1, len(self.array_of_w)):
                w = self.array_of_w[i][0]
                c = self.array_of_w[i][1]
                w_avg = w_avg + c*w

        for i in range(len(self.X)):
            x = self.X.iloc[i]
            sign = self.perceptron_get_sign(w_avg, x)
            self.avg_predictions.append(sign)

        return self.avg_predictions
        
        

In [422]:
def getError(preds, y):
    return 1 - ((preds == y).sum() / len(y))

# Question 1: 
    What are the training errors and the test errors of perceptron, voted perceptron and averaged perceptron after two, three and four passes?
### First, will create an instances of Perceptrons with 2, 3, and 4 rounds
### Then fit the training data to each one of these
## the y labels have been mapped 1 -> -1 and 2 -> 1

In [428]:
# takes one argument for the number of rounds
cls2 = Perceptron(2)
cls3 = Perceptron(3)
cls4 = Perceptron(4)

In [429]:
cls2.fit(X_train_2, y_train_2_trans)
cls3.fit(X_train_2, y_train_2_trans)
cls4.fit(X_train_2, y_train_2_trans)

In [430]:
preds2 = cls2.predict()
preds3 = cls3.predict()
preds4 = cls4.predict()

#### Computing errors for perceptron

In [441]:
print("Errors for Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(preds2, y_train_2_trans), getError(preds3, y_train_2_trans), getError(preds4, y_train_2_trans)))





Errors for Perceptron:
Error for 2 rounds: 0.0357798165137615
Error for 3 rounds: 0.01834862385321101
Error for 4 rounds: 0.016513761467889854


### Get predictions for voted perceptron and compute errors

In [437]:
voted_pred2 = cls2.voted_predict()
voted_pred3 = cls3.voted_predict()
voted_pred4 = cls4.voted_predict()

In [442]:
print("Errors for Voted Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(voted_pred2, y_train_2_trans), getError(voted_pred3, y_train_2_trans), getError(voted_pred4, y_train_2_trans)))



Errors for Voted Perceptron:
Error for 2 rounds: 0.038532110091743066
Error for 3 rounds: 0.026605504587155937
Error for 4 rounds: 0.022018348623853212


### Get predictions for averaged perceptron and compute errors

In [443]:
avg_preds2 = cls2.avg_predict()
avg_preds3 = cls3.avg_predict()
avg_preds4 = cls4.avg_predict()

In [444]:
print("Errors for Averaged Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(avg_preds2, y_train_2_trans), getError(avg_preds3, y_train_2_trans), getError(avg_preds4, y_train_2_trans)))




Errors for Averaged Perceptron:
Error for 2 rounds: 0.05137614678899083
Error for 3 rounds: 0.034862385321100864
Error for 4 rounds: 0.031192660550458662


In [333]:
# fit
def perceptron_fit(X, y):
    w = pd.Series(np.arange(len(X.columns)))*0
    m = 1
    c = 1
    arr_of_w = []
    for i in range(len(X)):
        if y.iloc[i]*w.dot(X.iloc[i]) <= 0:
            w = w + y.iloc[i]*X.iloc[i]
            m = m + 1
            c = 1
            arr_of_w.append([w, c])
        else:
            c = c + 1
            arr_of_w[len(arr_of_w) - 1][1] = arr_of_w[len(arr_of_w) - 1][1] + 1

    return w, arr_of_w

In [334]:
def perceptron_check(w, x): 
    return -1 if w.dot(x) <= 0 else 1

def perceptron_predict(X, w):
    arr = []
    for i in range(len(X)):
        x = X.iloc[i]
        arr.append(perceptron_check(w, x))
    return arr

In [335]:
def perceptron_check_c(w, x, c):
    return c*perceptron_check(w, x)
    
def voted_perception_predict(X, arr_w):
    preds = []
    for i in range(len(X)):
        x = X.iloc[i]
        voted_sum = 0
        for w_c in arr_w:
            w = w_c[0]
            c = w_c[1]
            voted_sum = voted_sum + perceptron_check_c(w, x, c)
            
        pred = -1 if voted_sum <= 0 else 1
        preds.append(pred)

    return preds
    

In [336]:
def average_perception_predict(X, arr_w):
    preds = []
    w = arr_w[0][0]
    c = arr_w[0][1]
    combination_w = c*w
    for i in range(1, len(arr_w)):
            w = arr_w[i][0]
            c = arr_w[i][1]
            combination_w = combination_w + c*w
    
    for i in range(len(X)):
        x = X.iloc[i]
        sign = perceptron_check(combination_w, x)
        preds.append(sign)
        
    return preds
        
    
    # now do dot to get sign
    
        
    

In [337]:
w, arrs_w = perceptron_fit(X_train_2, y_train_2_trans)

In [338]:
preds = perceptron_predict(X_train_2, w)

In [339]:
1 - (preds == y_train_2_trans).sum() / len(y_train_2_trans)

0.038532110091743066

In [340]:
preds_voted = voted_perception_predict(X_train_2, arrs_w)

In [314]:
len(preds_voted)

1090

In [341]:
1 - (preds_voted == y_train_2_trans).sum() / len(preds)

0.06605504587155964

In [316]:
preds_avg = average_perception_predict(X_train_2, arrs_w)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165


In [327]:
1 - (preds_avg == y_train_2_trans).sum() / len(preds)

0.07706422018348624

In [5]:
#X_train_w2.iloc[0][:label]

In [6]:
w = 0
check = 0
for i in range(len(X_train_2) - 1):
    if check == 0:
        w = y_train_2_trans.iloc[i]*X_train_2.iloc[i][:label]
        check = check + 1
    else:
        
        if y_train_2_trans.iloc[i]*w.dot(X_train_2.iloc[i][:label]) <= 0:
            print("changing")
            w = w + y_train_2_trans.iloc[i]*X_train_2.iloc[i][:label]

changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
changing
c

In [155]:
x_train_2_nolabel = df_train_2.drop([818], axis=1)

In [229]:
def Perceptron(trainingInput, w):
    arr = []
    for i in range(len(trainingInput)):
        x = trainingInput.iloc[i][:label]
        #y = y_train_2_trans.iloc[i]
        arr.append(perceptron(w, x))
    return arr

In [230]:
preds = Perceptron(X_train_2, w)

In [231]:
y_train_2.value_counts()

2.0    547
1.0    543
Name: 819, dtype: int64

In [232]:
pd.Series(preds).value_counts()

 1    548
-1    542
dtype: int64

In [233]:
error = 1 - ((preds == y_train_2_trans).sum() / len(preds))

In [234]:
y_train_2_trans.value_counts()

-1    547
 1    543
Name: 819, dtype: int64

In [235]:
error

0.041284403669724745

In [204]:
pd.Series(preds).value_counts()

 1    1045
-1      45
dtype: int64

In [89]:
perceptron(w, X_train_2.iloc[i][:label])

1

In [ ]:
for i in range(1):
    if()

In [11]:
one = X_train.iloc[0]


In [9]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,810,811,812,813,814,815,816,817,818,819
0,0.0,1.0,3.0,0.0,4.0,0.0,0.0,0.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0.0,0.0,12.0,0.0,0.0,0.0,0.0,2.0,12.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,16.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
